# Time Series classification of Aerodynamics
The boundary layer on an airfoil can be in two states, Laminar or Turbulent. Visually this switch is quite easy to see, for the aerodynamics muggles, the turbulent boundary layer state with show up on the time series data from the microphone as very turbulent(or volatile if you are from the finance world). 
Other ways to solve this problem include
- Traditional signal processing tricks that can extract such a change over of course. 
- Bayesian switch point analysis with a flexible number of switch points would also work. 
- The auto-regressive type models<br>
But I wanted to try my hand at getting a time series model in torch working, so here we are. 

Principles learnt:
- Training data has to be very good!! Fix this before trying fancy stuff like ensembling etc... 
- write test train data loading as a loop less code = less bugs. 
- Display metrics for all batches in validation set. single batches don't converge enough to give good results. Not a corner worth cutting
- Keep training even when the accuracy metric flattens out. Much of the struggle I had was simply not training long enough, the accurancy had flattened out but the pre-thresholded values contintued to go further to the rails. This meant that on the test set that the nn was more robust. Overfitting was still not a problem as nn used here was a little bit underpowered to start with so early stopping was not really nessescary. Training for 500-600 Epochs was in the end what produced the best model. A special regularization for output smoothness didn't seem nessescary in this case. 
- plot_grad_flow can help understand the architecture decisions i.e exploding or vanishing gradients. 
- Even though I didn't use an ensemble, I learnt that you can make snapshot ensembles or perform and ensemble fo the weights them selves and thus just keep one model. You can also ensemble across model types. i.e. CNN + RNN. Didn't end up need this although the architecture was almost built here. 



In [ ]:
import torch 
import pickle
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.utils.data
from torch.autograd import Variable
from sklearn.preprocessing import StandardScaler
from matplotlib import animation, rc
rc('animation', html='jshtml')
%matplotlib inline
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from PIL import Image

from fastai.vision import *
from fastai.metrics import error_rate
from fastai.vision.transform import *
import posixpath

# Set up Model
Multple Layer LSTM to fully conencted.
Sigmoid on the out layer as it is a binary classification problem. 
11> Lstm became difficult to train and didn't have the time to implement LR slicing across layers
The resulting model tends to be slightly biased and therefore quite resistant to overfitting. There isn't huge amounts of good training data so it will do. 

In [ ]:
class Classifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=2):
        super(Classifier, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)
        
        self.Sig = nn.Sigmoid()
        
    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        
        #input.view(self.input_dim, self.batch_size, -1)
        lstm_out, self.hidden = self.lstm(input.view(-1,len(input),8))
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.Sig(self.linear(lstm_out[-1].view(len(input), -1)))
        return y_pred.view(-1)

#model = LSTM(lstm_input_size, h1, batch_size=num_train, output_dim=output_dim, num_layers=num_layers)


# Read in data
The features of the time series have been pre-processed to include the time series itself and the first 6 mel-spectrum coefficients for more information see Librosa MFCC. Note that I did not split the test train sets before processing these coefficients which may constitute peeking, However, I finally tested the algorithm on files that were completely independent, the model seems to generalize well. I believe that one the batch size we are talking about the statistics have converged to the population values so we won't face any problems

In [ ]:
def LoadData(): #just doing it as a function to clean up whats in scope/ 
    #training files deliberately chosen to have a good representation of the classes and tend to be towards the leading edge
    # Trailing edge microphones naturally tend to be turbulent the whole time. 
    df = pd.read_pickle("./NewSensorData/Sensor10")
    Split = 0.8
    SplitInd = int(Split * len(df))
    LabelList = ['ts',*range(7)]
    Input = torch.tensor(df[LabelList].values)
    Output = torch.tensor(df['Labels'].values)

    #load second file hacky way for now. Only two files needed for training set.  
    df3 = pd.read_pickle("./NewSensorData/Sensor11")

    Input3 = torch.tensor(df3[LabelList].values)
    Output3 = torch.tensor(df3['Labels'].values)

    df4 = pd.read_pickle("./NewSensorData/Sensor0")
    
    Input4 = torch.tensor(df4[LabelList].values)
    Output4 = torch.tensor(df4['Labels'].values)

    #Valid trainSplit
    InpTrain4, InpValid4 = Input4[:SplitInd], Input4[SplitInd:]
    OutTrain4, OutValid4 = Output4[:SplitInd], Output4[SplitInd:]


    #Valid trainSplit
    InpTrain, InpValid = Input[:SplitInd], Input[SplitInd:]
    OutTrain, OutValid = Output[:SplitInd], Output[SplitInd:]

    #Valid trainSplit
    InpTrain3, InpValid3 = Input3[:SplitInd], Input3[SplitInd:]
    OutTrain3, OutValid3 = Output3[:SplitInd], Output3[SplitInd:]

    #Concat two files. 
    InputTrain = torch.cat((InpTrain,InpTrain3,InpTrain4),0)
    InputValid = torch.cat((InpValid,InpValid3,InpValid4),0)
    OutputTrain = torch.cat((OutTrain,OutTrain3,OutTrain4),0)
    OutputValid = torch.cat((OutValid,OutValid3,OutValid4),0)

    #Train scaler on training data only. No peeking now!! 
    InpScaler = StandardScaler()
    InpScaler.fit(InputTrain)

    InputTrain = torch.tensor(InpScaler.transform(InputTrain))
    InputValid = torch.tensor(InpScaler.transform(InputValid))

    BatchSize = 200000
    #Into DataLoaders
    # Large Batch size seems to perform better. 
    TrainDataSet = torch.utils.data.TensorDataset(InputTrain, OutputTrain)
    Train = torch.utils.data.DataLoader(TrainDataSet,batch_size = BatchSize,num_workers = 1)

    ValidDataSet = torch.utils.data.TensorDataset(InputValid, OutputValid)
    Valid = torch.utils.data.DataLoader(TrainDataSet,batch_size = BatchSize,num_workers = 1)
    LenTrain = len(InpTrain)
    LenValid = len(InpValid)
    return Train, Valid, BatchSize, InpScaler, LabelList

In [ ]:
Train, Valid, BatchSize,InpScaler,LabelList = LoadData()

In [ ]:
#save out scaler for use in other notebooks. 
with open('scaler.pickle', 'wb') as file:
    pickle.dump(InpScaler,file)

# Initiate with Simple paralell architecture 

In [ ]:
model = Classifier(8, 8, batch_size=BatchSize, output_dim=1, num_layers=11)
model.cuda()

# Adam optimizer with Cosine Annealing
Achieved 92% without momentum, Cosine Annealing appears effective from training. 
Have to implement a LR finder to better set. Works for now, probably could achieve much faster training

In [ ]:
lr = 1.2e-2
#optimizer = torch.optim.ASGD(model.parameters(),lr)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,10,0.1*lr)

In [ ]:
# Training Loop 
%matplotlib inline
highest = 0
regular = 0
for k in range(400):
    
    running_loss= 0 
    corrects = 0
    #Train
    for inp, target in Train:

        optimizer.zero_grad()
        inp = inp.cuda()
        output = model.forward(inp.float())
        #train
        Thresholded = torch.gt(output.float(),torch.tensor([0.5]).float().cuda()).float()
        loss = nn.functional.binary_cross_entropy(output, target.float().cuda()) #+ regular*torch.nn.functional.mse_loss(Conv(Thresholded.reshape([1,1,output.shape[0]])),torch.zeros([1,1,output.shape[0]-1]).cuda()) 
        #loss = CrossEntropy(output, target.float().reshape(output.shape).cuda())
        loss.backward()
        optimizer.step()
        running_loss += loss/len(target)  
            
    if k % 1==0: 
        print("epoch{}".format(k))
        print("train loss: {}".format(running_loss))
        #print("regular: {}".format(regular*torch.nn.functional.mse_loss(Conv(output.reshape([1,1,output.shape[0]])),torch.zeros([1,1,output.shape[0]-1]).cuda())/len(target)))
    running_loss= 0 
    # Added the option of just running the Training step for the first few epochs.
    if k > -1:
        # Validate. 
        a = np.array([])
        b= np.array([])
        for inp, target in Valid:
            inp = inp.cuda()
            output = model.forward(inp.float())
            Thresholded = torch.gt(output,torch.tensor([0.5]).float().cuda()).float()
            #valid
            loss = nn.functional.binary_cross_entropy(output, target.float().reshape(output.shape).cuda()) #+ regular*torch.nn.functional.mse_loss(Conv(Thresholded.reshape([1,1,output.shape[0]])),torch.zeros([1,1,output.shape[0]-1]).cuda())
            #Track Epoch Loss
            running_loss += loss/len(target)
            a = np.append(a,(output>0.5).cpu().numpy()) 
            b = np.append(b,(target.reshape(output.shape)>0.5).numpy())
        
        #Print out. 
        if k % 1 ==0: 
            print("valid loss: {}".format(running_loss))
            print("accuracy: {} %".format((a==b).mean().item()*100))
            if ((a==b).mean().item()*100)> highest:
                highest = ((a==b).mean().item()*100)
                #Save out model that achieves best accuracy. 
                torch.save(model.state_dict(), './ModelSave3.pt')
                print("saved")
            plt.figure()    
            sns.heatmap(confusion_matrix(a,b))
            plt.show()
        if k % 20 ==0:
            torch.save(model.state_dict(), './ModelSave{}.pt'.format(k))
            
    print("Learning Rate:{}".format(scheduler.get_lr()))
    scheduler.step() 

# For evaluating nn architecture 


In [ ]:
from matplotlib.lines import Line2D
# This code was copied from a blog. 
def plot_grad_flow(named_parameters):
    '''Plots the gradients flowing through different layers in the net during training.
    Can be used for checking for possible gradient vanishing / exploding problems.
    
    Usage: Plug this function in Trainer class after loss.backwards() as 
    "plot_grad_flow(self.model.named_parameters())" to visualize the gradient flow'''
    ave_grads = []
    max_grads= []
    layers = []
    for n, p in named_parameters:
        if(p.requires_grad) and ("bias" not in n):
            layers.append(n)
            ave_grads.append(p.grad.abs().mean())
            max_grads.append(p.grad.abs().max())
    plt.bar(np.arange(len(max_grads)), max_grads, alpha=0.1, lw=1, color="c")
    plt.bar(np.arange(len(max_grads)), ave_grads, alpha=0.1, lw=1, color="b")
    plt.hlines(0, 0, len(ave_grads)+1, lw=2, color="k" )
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(left=0, right=len(ave_grads))
    plt.ylim(bottom = -0.001, top=0.02) # zoom in on the lower gradient regions
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    plt.grid(True)
    plt.legend([Line2D([0], [0], color="c", lw=4),
                Line2D([0], [0], color="b", lw=4),
                Line2D([0], [0], color="k", lw=4)], ['max-gradient', 'mean-gradient', 'zero-gradient'])
plot_grad_flow(model.named_parameters())

# Evaluate Model
Run the RNN model over the results on a new file. 

Define bandswitching function that switches essentially only when it hits rails. Makes each state sticky. Was less important in well trained model

In [ ]:
def bandSwitch(window):
    if window[-1]>0.7:
        return 1
    elif window[-1]<0.3:
        return 0
    else: 
        return float('NaN')

In [ ]:
model2 = Classifier(8, 8, batch_size=BatchSize, output_dim=1, num_layers=11)
model2.load_state_dict(torch.load('./ModelSave3.pt'))
model2.cuda()
for k in range(47):
    # Get new data. 
    df = pd.read_pickle("./NewSensorData/Sensor{}".format(k))

    #Load into Torch Dataset. 
    Input = torch.tensor(InpScaler.transform(df[['ts',0,1,2,3,4,5,6]].values))
    Output = torch.tensor(df['Labels'].values)
    #Data -> Torch objects
    MainDataSet = torch.utils.data.TensorDataset(Input, Output)
    Main = torch.utils.data.DataLoader(MainDataSet,batch_size = 200000,num_workers = 1)
    full_results = torch.Tensor()
    #Run Model over the batches. 
    for inp, Label in Main:
        inp = inp.cuda()
        #maybe faster to turn off grads in future. Performance not a concern here. 
        output = model2.forward(inp.float())

        full_results = torch.cat((full_results,output.cpu()),0)
    df['rnnLabs'] = full_results.cpu().detach().numpy()
    #create Filtered Labels
    df['rnnFiltered'] = df['rnnLabs'].rolling(2).mean().rolling(2).apply(bandSwitch,raw=True).fillna(method='ffill').fillna(method='bfill')
    df['LabelsFiltered'] = df['Labels'].rolling(2).mean().rolling(2).apply(bandSwitch,raw=True).fillna(method='ffill').fillna(method='bfill')

    df.to_pickle("./Processed/Sensor{}".format(k))

In [ ]:
%matplotlib notebook
df[['ts','Labels','rnnLabs','rnnFiltered']].plot()

## CNN ---- Still using the not very well implemented input data. 
For Demonstration purposes. This code does not perform well. !!!
Input data still Melspectrum reduced with POD. Will probably perform as well as RNN with better input data. 

In [ ]:
#create picture of spectrum for a single df group 
# Save the file name with a random name and store the relationship in df
def pictureFromWindow(g):
    #print(g)
    ind = g[1].index[3]
    Lab = np.random.randint(0,1e15)
    #print(df.loc[g][['ts',0,1,2,3,4,5]])
    im = Image.fromarray(np.uint8((g[1][['ts',0,1,2,3,4,5]].values+1)*125))
    #print("./Pictures/{}/{}/{}.jpeg".format(case,Lab,next(p)))
    im.save("./Pictures/{}/{}.jpeg".format("test",Lab))
    return Lab,ind

Loop over time each 7 time steps being a group

In [ ]:
#p = iter(range(1000000000))
grouped = df.groupby(np.arange(len(df.index))//7)
df['Pictures'] = np.nan
for group in grouped:
    if group[1].shape[0]==7:
        Lab, ind = pictureFromWindow(group)
        df.loc[ind,'Pictures'] = int(Lab)
#df['Pictures'] = df['Pictures'].astype('int').interpolate(method='nearest')
#df['Pictures'] = df['Pictures']

In [ ]:
short = df.dropna()
short['PicturePath'] = short['Pictures'].apply(lambda x : "/test/{}.jpeg".format(int(x)))
short.rename(columns ={'LabelsFiltered':'label','PicturePath':'name'})[['name','label']]

In [ ]:
data = ImageDataBunch.from_df('./Pictures', short , size=7).normalize(imagenet_stats)

In [ ]:
df['NNLabel'] = full_results.detach().numpy()
df['Filtered'] = df['NNLabel'].rolling(10).mean().rolling(2).apply(bandSwitch)
df= df.fillna(method='ffill').fillna(method='bfill')

In [ ]:
#Time Series
%matplotlib inline
plt.figure(figsize=(20,10))
#plt.plot(full_results[:,-1].detach().numpy())
#plt.plot(full_results[:,-2].detach().numpy())

plt.plot(df['ts'].values[:])
plt.plot(df['NNLabel'].values[:],'k')
plt.plot(df['Filtered'].values[:],'r')
plt.plot(df['Labels'].values[:],'y')

plt.show()

In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate
from fastai.vision.transform import *
import posixpath

In [ ]:
bs = 2056
data = ImageDataBunch.from_folder("./Pictures/", size=7, bs=bs).normalize(imagenet_stats)

In [ ]:
learn = create_cnn(data, models.resnet50, metrics=error_rate)
learn.load('stage-1-50')

In [ ]:
predictions = [learn.predict(img)[2][1].numpy().item() for img in learn.data.train_ds.x[:30000]]

In [ ]:
paths = [int(posixpath.basename(posixpath.splitext(item)[0])) for item in data.train_ds.items]

In [ ]:
dfPredict = pd.DataFrame({"real":data.train_ds.y.items[:30000],"nnLabel":predictions[:30000],"path":paths[:30000]})

In [ ]:
dfPredict.set_index("path",inplace=True)

In [ ]:
def bandSwitch(window):
    if window[-1]>0.9:
        return 1
    elif window[-1]<0.1:
        return 0
    else: 
        return float('NaN')
        

In [ ]:
dfPredict.sort_index(inplace=True)

In [ ]:
dfPredict['Filtered'] = dfPredict['nnLabel'].rolling(2).mean().rolling(2).apply(bandSwitch)

In [ ]:
dfPredict= dfPredict.fillna(method='ffill').fillna(method='bfill')

In [ ]:
%matplotlib inline
plt.figure(figsize=(20,10))
plt.plot(dfPredict["real"].values,lw=10)
plt.plot(dfPredict["nnLabel"].values,color = 'r')
plt.plot(dfPredict["Filtered"].values,color = 'k')